In [1]:
from bs4 import BeautifulSoup
import pandas as pd
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document

In [3]:
# mainPath1 = './templates/contents/contents_1/귀어귀촌개념.html'
# mainPath2 = './templates/contents/contents_1/귀어귀촌준비절차.html'
# mainPath3 = './templates/contents/contents_1/어업면허허가.html'
mainPath4 = './templates/contents/contents_5/어업종류.html'

# mainPath5 = './templates/contents/contents_3/이상생태현상_굴폐사.html'
# mainPath6 = './templates/contents/contents_3/이상생태현상_산호초백화.html'
# mainPath7 = './templates/contents/contents_4/IoTAI활용스마트양식기술소개.html'

file_path = [
            # mainPath1
            # ,mainPath2
            # ,mainPath3
            mainPath4
            # ,mainPath5
            # ,mainPath6
            # ,mainPath7
             ]
contentList = []

splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,     # 총 길이
    chunk_overlap=50,   # 겹치는 길이
    separators=["\n\n", "\n", ".", " ", ""],  # 끊는 우선순위
)



for i in file_path:
    with open(i, 'r', encoding='utf-8') as f:
        html_content = f.read()
    
    folder = os.path.basename(os.path.dirname(i)) 
    page = os.path.splitext(os.path.basename(i))[0]  
    soup = BeautifulSoup(html_content, 'html.parser')
    sections = soup.select('[name="section"]')
    titles = None
    contents = None
    
    title = None
    content = None
    for j in sections:
        titles = j.select('[name="content_title"]')
        contents = j.select('[name="content_text"]')
        # 둘 다 1개 이상일 때
        if len(titles) > 1 and len(contents) > 1:
            count = min(len(titles), len(contents))  # 길이 맞추기
            for k in range(count):
                title = titles[k].get_text(strip=True)

                content = contents[k].get_text(strip=True)
                

                chunks = splitter.split_text(content)
                for k, chunk in enumerate(chunks):
                    contentList.append([title, chunk, folder, page])

        # 하나만 있을 경우 (정확히 하나일 때만 출력)
        elif len(titles) == 1 and len(contents) == 1:
            title = titles[0].get_text(strip=True)
            content = contents[0].get_text(strip=True)
            chunks = splitter.split_text(content)
            for k, chunk in enumerate(chunks):
                contentList.append([title, chunk, folder, page])
        else:
            print("제목 또는 내용이 없습니다.")
            # print(j)

df = pd.DataFrame(contentList,columns=["title", "content", "folder", "page"])
df



,title,content,folder,page
0,맨손 어법,"수산 동·식물을 직접 손으로 또는 간단한 도구로 잡거나, 훈련된 동물(개, 수달, ...",contents_5,어업종류
1,맨손 어법,각종 잠수장비를 착용하고 배로부터 공기를 공급받으면서 조업하는 잠수기 어업과 공기의...,contents_5,어업종류
2,손으로 더듬어 잡는 것(Grasping by hand),우리나라에서 행하고 있는 대표적인 것으로는 조개류 채취어업과 손꽁치 어업 등이 있다...,contents_5,어업종류
3,잠수하여 잡는것(By diving),"사람이 직접 물 속으로 들어가 대상물을 확인한 다음 칼, 갈고리 등 간단한 도구를 ...",contents_5,어업종류
4,살상어구류,수산 동·식물을 대상으로 손잡이가 달린 도구를 직접 손에 잡고 대상물을 잡거나 끝이...,contents_5,어업종류
5,살상어구류,. 따라서 대부분 수용기구류와 같이 대상물을 확인한 다음 채포한다. 우리나라에서 소...,contents_5,어업종류
6,"수용기구류(手用器具類, Hand instruments)","일정한 장소에 정착하여 서식하는 극피동물, 조개류, 해조류 등을 대상으로 긴 손잡이...",contents_5,어업종류
7,"투사물류(投射物類, Sharp projectiles)",끝이 날카로운 도구를 직접 대상물을 향해 던지거나 쏘아서 잡는 것이다. 따라서 대부...,contents_5,어업종류
8,마비어구류,수산동물을 마비시켜 도망가지 못하게 한 다음 채그물류 등으로 떠서 잡는 것이다. 마...,contents_5,어업종류
9,마비어구류,".화학적 마비어구류(化學的 痲痺具類, Chemical stupefying)유독성 화...",contents_5,어업종류


In [45]:
# 2. 문서 객체 생성
documents = [
    Document(page_content=row['content'], metadata={"title": row['title'], "folder": row['folder'], "page": row['page']})
    for _, row in df.iterrows()
]

embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")

vectorstore = FAISS.from_documents(documents=documents, embedding=embedding_model)

vectorstore.save_local("faiss_store/contents")

C:\Users\human\AppData\Local\Temp\ipykernel_5956\4096798401.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sroberta-multitask")
